In [4]:
import pandas as pd
import sympy as sp
import math as mt
import numpy as np

In [ ]:
def truncar(numero, decimales):
    """   
    :numero: El número a truncar
    :param decimales: Cuántos decimales mantener
    :return: número truncado en número n de decimales, ejemplo: original= 0.335, 0.33 a 2 decimales truncados
    """
    factor = 10 ** decimales
    return mt.trunc(numero * factor) / factor
def coeficientes_minimos_cuadrados_general(func_str, x_lista, y_lista, valores_iniciales):
    """
    :param func_str: string, función base para el modelado
    :param x_lista: lista de valores de x
    :param y_lista: lista de valores de y
    :param valores_iniciales: lista con valores iniciales, ej: [1, 0.1]
    :return: diccionario {a: valor_est, b: valor_est, ...}
    """
    f = sp.sympify(func_str)

    simbolos = list(f.free_symbols) #Obtiene las variables independientes y coeficientes de la función

    x = sp.Symbol('x')
    if x not in simbolos:
        raise ValueError("La función debe contener el símbolo 'x' como variable independiente.")
    
    parametros = [s for s in simbolos if s != x] #Filtra y guarda únicamente los coeficientes

    if len(parametros) != len(valores_iniciales):
        raise ValueError("El número de valores iniciales debe coincidir con el número de parámetros.")

    S = 0
    #Lógica para las derivadas parciales y resolucion del sistema de ecuaciones
    for xi, yi in zip(x_lista, y_lista):
        S += (yi - f.subs(x, xi))**2

    ecuaciones = [sp.diff(S, p) for p in parametros]

    solucion = sp.nsolve(
        ecuaciones,
        parametros,
        valores_iniciales,
        tol=1e-8
    )

    return {str(parametros[i]): float(solucion[i]) for i in range(len(parametros))} #Crea diccionario con las soluciones

def minimos_cuadrados_general_simbolico(func_str, coeficientes, decimales):
    """
    :param func_str: string, función original
    :param coeficientes: diccionario, valor de los coeficientes de la función
    :param decimales: entero, número de decimales a truncar
    :return: string con la función y sus respectivos coeficientes truncados según decimales
    """

    # Ciclo for para reemplazar los coeficientes en la funcion original
    for clave, valor in coeficientes.items():
        func_str = func_str.replace(str(clave), str(truncar(valor, decimales)))   
    return func_str

def minimos_cuadrados_general_numerico(func_str, coeficientes):
    """
    :param func_str: string, función original
    :param coeficientes: lista con los coeficientes del modelo
    :return: función lambda, capaz de realizar operaciones del tipo f(arg), donde arg es un valor y f devuelve otro
    """

    # Definicion de la variable simbolica y sustitucion de los coeficientes
    x = sp.Symbol("x")
    func_num = sp.sympify(func_str)
    func_num = func_num.subs(coeficientes)
    return sp.lambdify(x, func_num, "numpy")

def crear_modelo(entrada, modelo_base):

# Validacion de la lectura del archivo CSV
    try:
        df = pd.read_csv("datos_ajuste.csv")
    except FileNotFoundError:  # Archivo no encontrado
        print("Archivo no encontrado:", entrada)  # Mensaje de error
        return None
    except Exception as e:  # Captura de otro tipo de errores
        print("Error al leer el CSV:", e)  # Mensaje de error
        return None
    
    # Preparacion de los datos para el ajuste
    datos_x = [i + 1 for i in range(len(df))]
    datos_y = df['sales_week'].tolist()
    valores_iniciales = [0] * len([s for s in list(sp.sympify(modelo_base).free_symbols) if s !=sp.Symbol('x')])
    coeficientes = coeficientes_minimos_cuadrados_general(modelo_base, datos_x, datos_y, valores_iniciales)
    return minimos_cuadrados_general_numerico(modelo_base, coeficientes), minimos_cuadrados_general_simbolico(modelo_base, coeficientes, 5)
_, modelo_1 =crear_modelo("datos_ajuste.csv", "a*x+b")
_, modelo_2 = crear_modelo("datos_ajuste.csv", "a*x**2+b*x+c")
print(modelo_1, modelo_2)

4957.04751*x+313743.88309 -2.41086*x**2+5424.75467*x+298543.40035


In [ ]:
def metricas_modelo(entrada, modelo_base, valores_iniciales=None):
    try:
        df = pd.read_csv(entrada)
    except FileNotFoundError:
        print("Archivo no encontrado:", entrada)
        return None
    except Exception as e:
        print("Error al leer el CSV:", e)
        return None

    # y: datos reales
    y = pd.to_numeric(df["sales_week"], errors="coerce").dropna().to_numpy(dtype=float)
    n = len(y)
    x = np.arange(n, dtype=float)
    # num_parametros se trata del numero de parametros del modelo
    num_parametros = len([s for s in list(sp.sympify(modelo_base).free_symbols) if s != sp.Symbol('x')])
    if valores_iniciales is None:
        valores_iniciales = [0] * num_parametros
    if len(valores_iniciales) != num_parametros:
        print("El número de valores iniciales no coincide con el número de parámetros.")
        return None
        
    f, _ = crear_modelo(entrada, modelo_base)
    
    # y_sombrero: predicción del modelo en los mismos x de los datos
    y_sombrero = np.array(f(x), dtype=float)
    ## Error del ajuste
    residuo = y - y_sombrero # Diferencia entre valor real y estimado
    sse = float(np.sum(residuo**2)) # Suma de errores cuadráticos
    mse = float(sse / n) # Error cuadrático medio
    rmse = float(np.sqrt(mse)) # Raíz del error cuadrático medio
    mae = float(np.mean(np.abs(residuo))) # Error absoluto medio
    ## Calidad del ajuste
    sst = float(np.sum((y - np.mean(y))**2)) # Variabilidad total de los datos
    r2 = float(1.0 - sse/sst) if sst > 0 else float("nan")  # Coeficiente de determinación
    ## Complejidad del modelo
    cte = 1e-12 # Constante que evita log(0)
    aic = float(n * np.log((sse / n) + cte) + 2 * (num_parametros + 2)) # Criterio AIC
    bic = float(n * np.log((sse / n) + cte) + (num_parametros + 2) * np.log(n)) # Criterio BIC
    return {"RMSE": rmse, "MAE": mae, "R2": r2, "AIC": aic, "BIC": bic}
    
print("Modelo lineal: ", metricas_modelo("datos_ajuste.csv", "a*x+b"))
print("Modelo cuadrático: ", metricas_modelo("datos_ajuste.csv", "a*x**2+b*x+c"))


Modelo lineal:  {'RMSE': 195660.39772971568, 'MAE': 154385.062841357, 'R2': 0.6657409390323212, 'AIC': 4711.076407516762, 'BIC': 4724.127168272381}
Modelo cuadrático:  {'RMSE': 195546.13502336617, 'MAE': 154490.66461695812, 'R2': 0.6661312294866369, 'AIC': 4712.850923521928, 'BIC': 4729.1643744664525}


In [ ]:
def g(x, meta):
    return 4957.04751*x + 313743.88309 - meta

In [8]:
def biseccion(meta, a, b, tol, Max):
    """
    Parametros:
    meta: valor de ventas objetivo
    a: limite inferior
    b: limite superior
    tol: toleracia
    Max: Maximo de iteraciones
    Return:
    c: raiz aproximada
    """
    # Verificacion de cambio de signo en el intervalo
    if g(a, meta) * g(b, meta) >= 0: # Si el produto de las funciones es mayor o igual a 0 no existe cambio de signo
        print("No existe cambio de signo en el intervalo") # Mensaje de error
        return None
    
    iteraciones = 0 
    # Verificacion de intervalo para aplicar la formula de biseccion ((b-a)/2 >tol)
    while (b -a) / 2 > tol and iteraciones < Max: 
        c = (a + b) / 2
        # Comparacion de valores para la raiz encontrada
        if g(c, meta) == 0: # Si el valor es 0 se retorna la raiz
            return c
            # Si el valor es negativo se actualiza el limite superior
        elif g(c, meta) * g(a, meta) < 0: 
            b = c
            # Si el valor es positivo se actualiza el limite inferior
        else:
            a = c
        
        iteraciones += 1 # Incremento del numero de iteraciones
    return (a + b) / 2 # Retorno de la raiz aproximada
    



In [ ]:
df_val = pd.read_csv("datos_validacion.csv")

print("="*70)
print("VALIDACIÓN DEL MÉTODO DE BISECCIÓN")
print("="*70)
print(f"{'Meta Ventas':>15} | {'Semana Bisección':>17} | {'Semana Real':>12} | {'APE (%)':>10}")
print("-"*70)

# Intervalo de búsqueda: semanas 195 a 243 (validación)
a, b = 195, 243
tol = 10e-3  # Tolerancia de 0.01
max_iter = 100

# Creacion de lista para guardar errores
errores_porcentaje = []

# Iteracion sobre los datos de validacion
for idx in range(len(df_val)):
    meta_ventas = df_val['sales_week'].iloc[idx]
    semana_real = 195 + idx
    
    semana_biseccion = biseccion(meta_ventas, a, b, tol, max_iter)
    
    if semana_biseccion is not None:
        # APE = |real - estimado| / real * 100
        ape = abs(semana_real - semana_biseccion) / semana_real * 100
        errores_porcentaje.append(ape)
        
        # Muestra de algunas filas para no saturar la salida
        if idx in [0, 10, 20, 30, 48]:
            print(f"{meta_ventas:>15,.0f} | {semana_biseccion:>17.3f} | {semana_real:>12} | {ape:>9.4f}%")
    else:
        if idx in [0, 10, 20, 30, 48]:
            print(f"{meta_ventas:>15,.0f} | {'N/A':>17} | {semana_real:>12} | {'N/A':>10}")

# Calculo del rango MAPE
if errores_porcentaje:
    mape = sum(errores_porcentaje) / len(errores_porcentaje)
    print("="*70)
    print(f"\nMAPE (Mean Absolute Percentage Error): {mape:.4f}%")
    print(f"Número de predicciones exitosas: {len(errores_porcentaje)} de {len(df_val)}")

VALIDACIÓN DEL MÉTODO DE BISECCIÓN
    Meta Ventas |  Semana Bisección |  Semana Real |    APE (%)
----------------------------------------------------------------------
No existe cambio de signo en el intervalo
      1,084,355 |               N/A |          195 |        N/A
No existe cambio de signo en el intervalo
No existe cambio de signo en el intervalo
No existe cambio de signo en el intervalo
No existe cambio de signo en el intervalo
No existe cambio de signo en el intervalo
No existe cambio de signo en el intervalo
No existe cambio de signo en el intervalo
      1,272,342 |               N/A |          205 |        N/A
No existe cambio de signo en el intervalo
No existe cambio de signo en el intervalo
No existe cambio de signo en el intervalo
      1,362,405 |           211.553 |          215 |    1.6034%
No existe cambio de signo en el intervalo
No existe cambio de signo en el intervalo
No existe cambio de signo en el intervalo
      1,312,699 |           201.521 |          225